# **Modelo 2 (Generacion de Poemas) GPT2**


---

In [1]:
pip install transformers -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Basicas
import pandas as pd
import numpy as np
import os
import time
import datetime
# Pytorch
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler
# Texto
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import get_scheduler
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
from transformers import pipeline
# Sklearn
from sklearn.model_selection import train_test_split
# Funciones y variables
def format_time(elapsed): return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [3]:
# Informacion del modelo ------------------------------------------------------\
max_length = 800 # Longitud maxima de los poemas
modelo_gpt = "DeepESP/gpt2-spanish" # Modelo pre entrenado
RANDOM_SEED = 2022 # Semilla

# **Datos**

In [4]:
url = 'https://raw.githubusercontent.com/andreamorgar/poesIA/master/data/poems.csv'
poems_df = pd.read_csv(url)
poems_df = poems_df.dropna()

In [5]:
# Filtrar poemas grandes
poems_df['string'] = poems_df.apply(lambda row: f'\n{row["title"]}\n{row["content"]}', axis=1)
poems_df['length'] = poems_df.string.map(len)
poems_filtered = poems_df[poems_df.length < max_length]
_ , poems_filtered = train_test_split(poems_filtered, test_size = 0.9 ,shuffle=True,random_state = 2022)
poems_filtered

,author,content,title,string,length
2544,Francisco de Figueroa,"\n\nPerdido ando, señora, entre la gente,\nsin...",SONETO XVII,"\nSONETO XVII\n\n\nPerdido ando, señora, entre...",558
908,Luis de Góngora,"\n\n¡Oh, de alto valor, de virtud rara\nSacro ...",A DON ANTONIO VENEGAS,"\nA DON ANTONIO VENEGAS\n\n\n¡Oh, de alto valo...",511
5029,José Gautier Benítez,\n\nCuando no reste ya ni un solo grano\nde mi...,A MIS AMIGOS,\nA MIS AMIGOS\n\n\nCuando no reste ya ni un s...,599
3942,Víctor Jiménez,"A la ausencia, al olvido, a la nostalgia\nmi c...",Tango para engañar a la tristeza,\nTango para engañar a la tristeza\nA la ausen...,272
3927,Gabriel García Márquez,"Si alguien llama a tu puerta, amiga mía,\ny al...",Si alguien llama a tu puerta,\nSi alguien llama a tu puerta\nSi alguien lla...,544
...,...,...,...,...,...
479,Jorge Teillier,\n\nSentados frente al fuego que envejece\nmir...,SENTADOS FRENTE AL FUEGO,\nSENTADOS FRENTE AL FUEGO\n\n\nSentados frent...,703
4052,Juan Ramón Jiménez,\n\n¡Qué miedo el azul del cielo!\n¡Negro!\n¡N...,TRASCIELO DEL CIELO AZUL,\nTRASCIELO DEL CIELO AZUL\n\n\n¡Qué miedo el ...,267
299,Víctor Botas,No me preguntes cómo pasa el tiempo\nLi Kiu Li...,Las rosas de Babilonia,\nLas rosas de Babilonia\nNo me preguntes cómo...,703
2065,Toni García Arias,Decías unas cosas que me asustaban.\nEn cubier...,Sobre la cubierta,\nSobre la cubierta\nDecías unas cosas que me ...,372


In [6]:
print(list(poems_filtered['string'])[20])


Llegué a Valladolid; registré luego


Llegué a Valladolid; registré luego
Desde el bonete al clavo de la mula;
Guardo el registro, que será mi bula
Contra el cuidado del señor don Diego.

Busqué la Corte en él, y yo estoy ciego,
O en la ciudad no está, o se disimula.
Celebrando dïetas vi a la gula,
Que Platón para todos está en griego.

La lisonja hallé y la ceremonia
Con luto, idolatrados los caciques,
Amor sin fe, interés con sus virotes.

Todo se halla en esta Babilonia,
Como en botica, grandes alambiques,
Y más en ella títulos que botes.


# **Tokens para los datos (modelo DeepESP/gpt2-spanish)**

In [7]:
df = poems_filtered['string'] # Datos
# Tokenizador del modelo pre entrenado ----------------------------------------\
tokenizer = GPT2Tokenizer.from_pretrained(modelo_gpt)
special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [8]:
# Tokenizador del modelo ------------------------------------------------------\
class DataTokens(Dataset):
  def __init__(self, data, tokenizer, gpt2_type="gpt2", max_length=max_length):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    for row in data:
      self.encodings_dict = self.tokenizer('<BOS>' + row + '<EOS>', padding="max_length", truncation=True, max_length=max_length)
      self.input_ids.append(torch.tensor(self.encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(self.encodings_dict['attention_mask']))
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]
# Clase de los datos ----------------------------------------------------------\
class DataModule():
  # Definimos un tamaño de lote en la clase
  def __init__(self, dataset, tokenizer, gpt2_type="gpt2", p = 0.8):
      super(DataModule,self).__init__()
      self.dataset = dataset
      self.tokenizer = tokenizer
      self.p = p
      self.gpt2_type = gpt2_type
  # Definimos el tratamiento de los datos
  def train_val_split(self, split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size
  def setup(self, stage=None):
    self.dataset = DataTokens(self.dataset, self.tokenizer, gpt2_type=self.gpt2_type)
    train_size, val_size = self.train_val_split(self.p, self.dataset)
    self.train_dataset, self.val_dataset = random_split(self.dataset, [train_size, val_size])
  # Iterable de entrenamiento
  def train_dataloader(self, batch_size = 32):
      return torch.utils.data.DataLoader(self.train_dataset, batch_size=batch_size)
  # Iterable de validacion
  def val_dataloader(self, batch_size = 32):
      return torch.utils.data.DataLoader(self.val_dataset, batch_size=batch_size)

# **Reentrenamiento para el modelo (DeepESP/gpt2-spanish)**

In [9]:
# Fijar semillas --------------------------------------------------------------\
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [37]:
#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor[0].size()[1] + packed_tensor[0].size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = [torch.cat([new_tensor[0], packed_tensor[0][:, 1:]], dim=1)
          ,torch.cat([new_tensor[1], packed_tensor[1][:, 1:]], dim=1)]
        return packed_tensor, True, None
# Entrenamiento del modelo ----------------------------------------------------\
class Trainer_poet():
    def __init__(self, dataset, model, batch_size=16, epochs=5, learning_rate = 1e-4, eps = 1e-8, warmup_steps=50):
      # DataLoaders
      self.data_loader = dataset
      self.data_loader.setup()
      self.train_dataloader = self.data_loader.train_dataloader(batch_size = 1)
      self.val_dataloader = self.data_loader.val_dataloader(batch_size = 1)
      # Modelo
      self.model = model
      self.batch_size = batch_size
      self.epochs = epochs
      self.optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)
      total_steps = len(self.train_dataloader) * epochs
      self.scheduler = get_linear_schedule_with_warmup(optimizer=self.optimizer,num_warmup_steps=warmup_steps,num_training_steps=total_steps)
    def train(self):
      device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
      model = self.model
      optimizer =  self.optimizer
      scheduler = self.scheduler
      model.to(device)
      model.train()
      start_time = time.time()
      training_stats = []
      # Entrenamiento
      print('Inicio entrenamiento ....')
      train_dataloader = self.train_dataloader
      val_dataloader = self.val_dataloader
      for epoch_i in range(self.epochs):
        print(f'Epoch {epoch_i + 1} de {self.epochs}')
        t0 = time.time()
        total_train_loss = 0
        input_tensor = None
        accumulating_batch_count = 0
        for step, batch in enumerate(train_dataloader):
          (input_tensor, carry_on, remainder) = pack_tensor(batch, input_tensor, 768)
          if carry_on and step != len(train_dataloader) - 1: continue

          b_input_ids = input_tensor[0].to(device)
          b_masks = input_tensor[1].to(device)
          input_tensor = [b_input_ids, b_masks]
          outputs = model(b_input_ids,labels=b_input_ids,attention_mask=b_masks)
          loss = outputs[0]
          loss.backward()
          if (accumulating_batch_count % self.batch_size) == 0:
            batch_loss = loss.item()
            total_train_loss += batch_loss
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()
          accumulating_batch_count += 1
          input_tensor = None
        avg_train_loss = total_train_loss / len(train_dataloader)
        training_time = format_time(time.time() - t0)
        print(f'Average Training Loss: {avg_train_loss}. Epoch Training Time: {training_time}')
        # Validacion
        t0 = time.time()
        model.eval()
        total_eval_loss = 0
        nb_eval_steps = 0
        for batch in val_dataloader:
          b_input_ids = batch[0].to(device)
          b_masks = batch[1].to(device)
          with torch.no_grad():
            outputs  = model(b_input_ids,attention_mask=b_masks,labels=b_input_ids)
            loss = outputs[0]
          batch_loss = loss.item()
          total_eval_loss += batch_loss
        avg_val_loss = total_eval_loss / len(val_dataloader)
        validation_time = format_time(time.time() - t0) 
        print(f'Average Validation Loss: {avg_val_loss}')
        # Guardar estadisticas
        training_stats.append(
            {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Training Time': training_time,
              'Validation Time': validation_time
             }
          )
      self.training_stats = training_stats
      self.model = model
      print(f'Total Training Time: {format_time(time.time()-start_time)}')
      return model

In [40]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions=max_length).from_pretrained(modelo_gpt, output_hidden_states=True)
model_gpt2_esp = GPT2LMHeadModel.from_pretrained(modelo_gpt, config=configuration)
model_gpt2_esp.resize_token_embeddings(len(tokenizer))

Dataset = DataModule(df, tokenizer, gpt2_type=modelo_gpt)
Trainer_model = Trainer_poet(Dataset, model_gpt2_esp, epochs=10, batch_size=16)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [41]:
model = Trainer_model.train()
torch.save(model, 'modelo_gpt2_poesia.pt')

Inicio entrenamiento ....
Epoch 1 de 10
Average Training Loss: 0.054826443357317636. Epoch Training Time: 0:02:55
Average Validation Loss: 0.7208625375721949
Epoch 2 de 10
Average Training Loss: 0.021068227584347933. Epoch Training Time: 0:02:51
Average Validation Loss: 0.6930688514702437
Epoch 3 de 10
Average Training Loss: 0.019718704691685334. Epoch Training Time: 0:02:51
Average Validation Loss: 0.6864213045642509
Epoch 4 de 10
Average Training Loss: 0.01870843443415455. Epoch Training Time: 0:02:51
Average Validation Loss: 0.6860056349011939
Epoch 5 de 10
Average Training Loss: 0.017727699577556672. Epoch Training Time: 0:02:51
Average Validation Loss: 0.68893584291157
Epoch 6 de 10
Average Training Loss: 0.016660080785309343. Epoch Training Time: 0:02:51
Average Validation Loss: 0.6952692445926683
Epoch 7 de 10
Average Training Loss: 0.015499611866559732. Epoch Training Time: 0:02:51
Average Validation Loss: 0.7046269372256369
Epoch 8 de 10
Average Training Loss: 0.01422815134975

# **Generación de Poesía**

In [ ]:
model = torch.load('modelo_gpt2_poesia.pt')

In [42]:
def generate(model,tokenizer,prompt,length=60,top_p=0.8,temperature=1.):
    #prompt = trad_es_en(prompt)[0]['translation_text']
    model.eval()
    generated_num = 0
    generated_list = []
    filter_value = -float("Inf")
    with torch.no_grad():
      entry_finished = False
      generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
      for i in range(length):
          outputs = model(generated, labels=generated)
          loss, logits = outputs[:2]
          logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)
          sorted_logits, sorted_indices = torch.sort(logits, descending=True)
          cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
          sorted_indices_to_remove = cumulative_probs > top_p
          sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
          sorted_indices_to_remove[..., 0] = 0
          indices_to_remove = sorted_indices[sorted_indices_to_remove]
          logits[:, indices_to_remove] = filter_value
          next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
          generated = torch.cat((generated, next_token), dim=1)
          if next_token in tokenizer.encode("<|endoftext|>"): break 
      output_list = list(generated.squeeze().numpy())
      output_text = tokenizer.decode(output_list,skip_special_tokens=True)
      generated_list.append(output_text)
    #generated_list = trad_en_es(generated_list)[0]['translation_text']  
    return generated_list[0]

In [51]:
Palabra = 'MI PEQUEÑA'
text = generate(model.to('cpu'), tokenizer,Palabra,temperature=0.7,length = 100,top_p = 0.8)
print(text)

MI PEQUEÑA


A Federico,
por quien me has tomado

el corazón,
por quien me has tomado
el corazón,
por quien me has tomado
el corazón,

por quien me has tomado
el corazón,

por quien me has tomado
el corazón,

por quien me has tomado
el corazón,


por quien me has tomado
el corazón,


por quien me has tomado
el corazón,


por quien me


In [66]:
def Generate2(model,Palabra):
  input_ids = tokenizer.encode(Palabra, return_tensors="pt")
  output = model.generate(
      input_ids,
      do_sample=True,
      top_k=50,
      max_length=200,
      top_p=0.95,
      num_return_sequences=1,
      #temperature=1.5
      #no_repeat_ngram_size=2,
      #early_stopping=True,
      #num_beams=5
  )
  output = tokenizer.decode(output[0], skip_special_tokens=True)
  return output

In [68]:
Palabra = 'MI MUERTE'
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


MI MUERTE

La tarde caía
los árboles derribados
en el polvo.

Conmovidos por el cansancio,
al alba, el Sena
se abría paso entre la bruma
derramando.

El agua, ya casi fría,
turba el agua de la Gran lluvia.

Se apagaba la lámpara
y, a cada nuevo relámpago,
el crepúsculo... 

Me hacía la oscuridad un sueño,
una agonía
cada nuevo heraldo,
cua que dijera en la noche
que yo era la noche. nunca... nunca...
Me ardían los oídos
la noche entera. siempre...
... una vez más, el mundo
en torno a mí se perdía,
y la noche, en el tiempo, una llama. siempre... nunca...



In [69]:
Palabra = 'TERMINAR'
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


TERMINAR


De su hermosura fina
la luz revela;
el alma olvida,
porque es luz de cielo;
y en vano la luz olvida.,

La hermosura me revela;
es luz de cielo,
de sol sin tasa fijo,
de sol sin tasa fijo
aún oscuro,
de luz a noche;

¿en vano la luz me revela,
y a qué precio?

Sucede que a tales cosas

No digo, señora, que no le amargue la luz,
que es gran bien;
mas quisiera yo a su hermosura
olvidarme,
mas quisiera a mi alma.,
la luz me revela;
y el alma me deja, me hace;
y la luz me da, y la luz me da.


In [72]:
Palabra = 'AGOTADO'
text = Generate2(model.to('cpu'),Palabra)
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


AGOTADO


Por ejemplo, si, y si
empuja,
no sé si se

tiende a tener fe,
si no la tiene,
no sé si se
cruza de los dedos.? (Nadie lo sabe. Es la voz de un muerto).

Pierde la mañana entera en la
conjunción de que fue muerta. Es la tristeza la que habla en el silencio. Y entonces me acuerdo del funeral de alguien que la vio morir... ¡la misma muerte!... la misma noche que la muerte... Le pregunté a la vieja, que me creía. Le respondieron que la muerta era la misma muerte. No puedo creer que haya sido tan triste... Le comenté a la vieja que no, pero que le dije que no era la muerte. Le dije que no, que era la muerte, sino la muerte. Le comenté que no. Le comenté que no
